In [1]:
# https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/nlp/ipynb/lstm_seq2seq.ipynb

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

c:\Users\Tyler\miniconda3\envs\ai-chatbot\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Tyler\miniconda3\envs\ai-chatbot\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\Users\Tyler\miniconda3\envs\ai-chatbot\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %
c:\Users\Tyler\miniconda3\envs\ai-chatbot\lib\site-packages\scipy\__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.18.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
batch_size = 32  # Batch size for training.
epochs = 150  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 5000  # Number of samples to train on.

In [4]:
# open file
df = pd.read_csv("../data/casual_data_windows.csv")

In [5]:
# Prepare data
df = df.drop([df.columns[0], df.columns[-1]], axis=1)

In [6]:
df = df.rename(columns={'0': 'query', '1': 'response'})

In [7]:
df = df[:num_samples]

In [8]:
df.head(50)

,query,response
0,What kind of phone(s) do you guys have?,I have a pixel. It's pretty great. Much better...
1,I have a pixel. It's pretty great. Much better...,Does it really charge all the way in 15 min?
2,Does it really charge all the way in 15 min?,"Pretty fast. I've never timed it, but it's und..."
3,What kind of phone(s) do you guys have?,Samsung Galaxy J1. It's my first cell phone an...
4,Samsung Galaxy J1. It's my first cell phone an...,What do you think of it? Anything you don't like?
5,What do you think of it? Anything you don't like?,I love it. I can't think of anything I don't l...
6,What kind of phone(s) do you guys have?,"LG Optimus V. I know, it's old."
7,"LG Optimus V. I know, it's old.","If it does it's job, it's good enough!"
8,My friend told me to kill myself :/,Don't kill yourself OP.
9,Don't kill yourself OP.,I won't give them the satisfaction ;)


In [9]:
df = df.replace({'\n': ' ', '\r': ''}, regex=True)

In [10]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for i, row in df.iterrows():
    input_text = row['query']
    target_text = row['response']
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [11]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [12]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [13]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [14]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [15]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

In [16]:

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


In [17]:

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))


In [18]:

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)


In [19]:

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [20]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")


Epoch 1/150
125/125 [==============================] - 111s 888ms/step - loss: 1.7287 - accuracy: 0.6613 - val_loss: 1.6841 - val_accuracy: 0.6273
Epoch 2/150
125/125 [==============================] - 110s 882ms/step - loss: 1.3953 - accuracy: 0.6682 - val_loss: 1.4268 - val_accuracy: 0.6326
Epoch 3/150
125/125 [==============================] - 114s 913ms/step - loss: 1.2452 - accuracy: 0.6828 - val_loss: 1.3030 - val_accuracy: 0.6572
Epoch 4/150
125/125 [==============================] - 118s 943ms/step - loss: 1.1308 - accuracy: 0.7018 - val_loss: 1.2098 - val_accuracy: 0.6737
Epoch 5/150
125/125 [==============================] - 119s 955ms/step - loss: 1.0577 - accuracy: 0.7154 - val_loss: 1.1646 - val_accuracy: 0.6830
Epoch 6/150
125/125 [==============================] - 119s 950ms/step - loss: 1.0092 - accuracy: 0.7237 - val_loss: 1.1505 - val_accuracy: 0.6871
Epoch 7/150
125/125 [==============================] - 117s 933ms/step - loss: 0.9758 - accuracy: 0.7299 - val_loss: 1

In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence



In [ ]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)


-
Input sentence: What kind of phone(s) do you guys have?
Decoded sentence: I love you don't kind of the mest atdang than somenom :) I cooll bee sliged now..

-
Input sentence: I have a pixel. It's pretty great. Much better than what I had before. 
Decoded sentence: I love you don't kind of the mest atdang than somenom :) I cooll bee sliged now..

-
Input sentence: Does it really charge all the way in 15 min?
Decoded sentence: I love you don't kind of the mest atdang than somenom :) I cooll bee sliged now..

-
Input sentence: What kind of phone(s) do you guys have?
Decoded sentence: I love you don't kind of the mest atdang than somenom :) I cooll bee sliged now..

-
Input sentence: Samsung Galaxy J1. It's my first cell phone and I've had it for 7 months.
Decoded sentence: I love you don't kind of the mest atdang than somenom :) I cooll bee sliged now..

-
Input sentence: What do you think of it? Anything you don't like?
Decoded sentence: I love you don't kind of the mest atdang than so